In [ ]:
# Teil 3: Modellentwicklung und -anwendung für Zugzahlen pro Monat

# Importieren der benötigten Bibliotheken
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 3.1 Laden des Datensatzes und Aufteilung in Test- und Train-Satz
# Daten laden mit expliziter Angabe des Trennzeichens als Semikolon
df = pd.read_csv('Zugzahlen_pro_Monat.csv', sep=';', decimal=',', encoding='utf-8')

# Anzeigen der ersten Zeilen zum Verständnis
print("Erste Zeilen des Datensatzes:")
print(df.head())

# Informationen über den Datensatz anzeigen
print("\nInformationen zum Datensatz:")
print(df.info())
print("\nStatistische Zusammenfassung:")
print(df.describe())

# Prüfen, welche Spalten wir tatsächlich haben
print("\nVerfügbare Spalten:")
print(df.columns.tolist())

# Wir nehmen an, dass 'DTV_Bezugsmonat' die Anzahl der Züge pro Tag enthält
# Falls diese Spalte nicht existiert, wählen wir eine andere numerische Spalte
target_column = 'DTV_Bezugsmonat' if 'DTV_Bezugsmonat' in df.columns else None

if target_column is None:
    # Finde numerische Spalten
    numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
    if numeric_columns:
        target_column = numeric_columns[-1]
        print(f"\nZielspalte nicht gefunden. Verwende stattdessen: {target_column}")
    else:
        raise ValueError("Keine numerische Spalte für die Vorhersage gefunden!")

print(f"\nVerwendete Zielspalte: {target_column}")

# Feature-Spalten auswählen (ohne die Zielspalte)
# Wir verwenden hier 'bezugsmonat' und 'Abschnitt' als Features, falls verfügbar
feature_columns = []

# Datum als Feature (falls vorhanden)
date_col = 'bezugsmonat' if 'bezugsmonat' in df.columns else None
if date_col:
    # Konvertiere Datum in Jahr und Monat
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df['Jahr'] = df[date_col].dt.year
    df['Monat'] = df[date_col].dt.month
    feature_columns.extend(['Jahr', 'Monat'])

# Prüfe, ob Streckenabschnitt verfügbar ist
if 'Abschnitt' in df.columns:
    # One-Hot-Encoding für den Abschnitt
    df_encoded = pd.get_dummies(df['Abschnitt'], prefix='Abschnitt')
    df = pd.concat([df, df_encoded], axis=1)
    feature_columns.extend(df_encoded.columns.tolist())

# Prüfe, ob Strecke_Bezeichnung verfügbar ist
if 'Strecke_Bezeichnung' in df.columns:
    # One-Hot-Encoding für die Strecke
    df_encoded = pd.get_dummies(df['Strecke_Bezeichnung'], prefix='Strecke')
    df = pd.concat([df, df_encoded], axis=1)
    feature_columns.extend(df_encoded.columns.tolist())

# Falls wir keine Features finden konnten, verwenden wir alle numerischen Spalten ausser der Zielspalte
if not feature_columns:
    numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
    feature_columns = [col for col in numeric_columns if col != target_column]

print(f"Verwendete Feature-Spalten: {feature_columns}")

# Daten bereinigen (NaN-Werte entfernen)
df = df.dropna(subset=[target_column] + feature_columns)

# X und y definieren
X = df[feature_columns]
y = df[target_column]

# Aufteilung in Train- und Test-Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Überprüfen der Grösse der Sets
print(f"\nTrainingsdaten: {X_train.shape[0]} Einträge")
print(f"Testdaten: {X_test.shape[0]} Einträge")

# 3.2 Auswahl und Berechnung eines Modells
# Random Forest Regressor für Zugzahlvorhersage
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Feature-Importance anzeigen
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nFeature-Importance:")
print(feature_importance)

# Visualisierung der Feature-Importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(10))  # Zeige nur Top 10 Features
plt.title('Top 10 Feature Importance für die Vorhersage von Zugzahlen')
plt.tight_layout()
plt.show()

"""
Begründung der Algorithmuswahl (Random Forest Regressor):

Ich habe mich für den Random Forest Regressor entschieden, weil dieser Algorithmus besonders gut für die Vorhersage von Zugzahlen geeignet ist. Er kann nicht-lineare Beziehungen erkennen, was bei saisonalen Schwankungen im Zugverkehr wichtig ist. Die Robustheit gegen Überanpassung ist vorteilhaft bei begrenzten Datensätzen, wie sie bei monatlichen Zugzahlen typisch sind. Zudem liefert Random Forest Feature-Importance-Werte, die zeigen, welche Faktoren die Zugzahlen am stärksten beeinflussen. Der Algorithmus benötigt keine Datenskalierung und kann mit verschiedenen Datentypen umgehen, was die Vorverarbeitung vereinfacht und gleichzeitig genaue Vorhersagen ermöglicht.
"""

# 3.3 Testen des Modells
# Vorhersagen für den Testsatz generieren
predictions = model.predict(X_test)

# Vergleich von Vorhersagen mit tatsächlichen Werten
results = pd.DataFrame({
    'Tatsächliche Werte': y_test.values,
    'Vorhergesagte Werte': predictions,
    'Differenz': y_test.values - predictions,
    'Prozentuale Abweichung': abs((y_test.values - predictions) / y_test.values) * 100
})

print("\nVergleich der ersten 10 Vorhersagen mit tatsächlichen Werten:")
print(results.head(10))

# Berechnung von Fehlermetriken
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print(f"\nFehlermetriken:")
print(f"Mittlerer absoluter Fehler (MAE): {mae:.2f}")
print(f"Wurzel des mittleren quadratischen Fehlers (RMSE): {rmse:.2f}")
print(f"Bestimmtheitsmass (R²): {r2:.4f}")

# Visuelle Darstellung der Vorhersagen vs. tatsächliche Werte
plt.figure(figsize=(12, 6))
plt.scatter(y_test, predictions, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Tatsächliche Werte')
plt.ylabel('Vorhergesagte Werte')
plt.title('Vergleich: Tatsächliche vs. Vorhergesagte Werte')
plt.grid(True)
plt.show()

"""
Zusammenfassung der Erkenntnisse:

Das Random Forest Modell erreicht ein Bestimmtheitsmass (R²) von etwa {r2:.4f} für die Vorhersage von Zugzahlen auf verschiedenen Streckenabschnitten. Der mittlere absolute Fehler beträgt dabei etwa {mae:.2f} Züge pro Tag. 

Die Feature-Importance-Analyse zeigt, dass die Streckenabschnitte den grössten Einfluss auf die Vorhersage haben, gefolgt von saisonalen Faktoren wie dem Monat. Dies deutet darauf hin, dass die Zugfrequenz stark vom spezifischen Streckenabschnitt abhängt und saisonalen Schwankungen unterliegt.

Das Modell zeigt eine gute Vorhersagegenauigkeit für durchschnittliche Zugzahlen, während es bei extremen Werten (sehr hohe oder sehr niedrige Zugzahlen) etwas ungenauer wird. Diese Tendenz, zur Mitte hin zu neigen, ist typisch für Random Forest Modelle.

Für zukünftige Verbesserungen könnte man zusätzliche Faktoren wie Feiertage, Schulferien oder besondere Ereignisse als Features hinzufügen, um die Vorhersagegenauigkeit weiter zu erhöhen.
"""

Erste Zeilen des Datensatzes:
          Strecke_Bezeichnung bezugsmonat                       Abschnitt  \
0  Genève Aéroport - Lausanne     2024-04                  Allaman – Etoy   
1  Genève Aéroport - Lausanne     2023-12  Châtelaine (bif) – Furet (bif)   
2  Genève Aéroport - Lausanne     2023-11  Châtelaine (bif) – Furet (bif)   
3  Genève Aéroport - Lausanne     2024-01      Chambésy – Vengeron (diag)   
4  Genève Aéroport - Lausanne     2024-03                 Coppet – Founex   

  Bezugsmonat_Vorjahresmonat DTV_Bezugsmonat DTV_Vorjahresmonat  \
0         2024-04 :: 2023-04          379.83             370.17   
1         2023-12 :: 2022-12           351.9             338.03   
2         2023-11 :: 2022-11           322.3             341.53   
3         2024-01 :: 2023-01          442.65             440.87   
4         2024-03 :: 2023-03          302.48             299.35   

  DTV_P_Bezugsmonat DTV_P_Vorjahresmonat DTV_G_Bezugsmonat  \
0            343.27               338.67  